In [2]:
import json
import re
from konlpy.tag import Okt
from collections import Counter
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc
from wordcloud import WordCloud
import pandas as pd

import glob
from functools import reduce
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

from wordcloud import WordCloud, STOPWORDS

import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
import gensim 
import warnings
warnings.filterwarnings('ignore')

# 데이터 정리
### 영화정보
> meta19 meta20 meta21

### 리뷰
> 네이버 : n19 | n20 | n21<br>
> 다음 : d19 | d20 | d21<br>
> 왓챠피디아 : w19 | w20 | w21

# 데이터 병합

In [1]:
import pandas as pd

In [10]:
meta19 = pd.read_csv('./data/movie/movie2019_last_of_last.csv')
meta20 = pd.read_csv('./data/movie/movie2020_last_of_last.csv')
meta21 = pd.read_csv('./data/movie/movie2021_last_of_last.csv')

In [18]:
meta192021 = pd.concat([meta19,meta20,meta21],axis=0)
meta192021.to_csv('./data/review/meta192021.csv', index=False)

In [5]:
n19 = pd.read_csv('./data/review/naver_review2019.csv')
n20 = pd.read_csv('./data/review/naver_review2020.csv')
n21 = pd.read_csv('./data/review/naver_review2021.csv')

In [27]:
n19.head()
n192021 = pd.concat([n19,n20,n21], axis=0)

In [28]:
n192021 = n192021.reset_index(drop=True)

# 전처리
* dropna (10,843개)
* 스포일러 리뷰 삭제 (9,599개)

In [30]:
n192021 = n192021.dropna()

In [31]:
n192021 = n192021[~n192021['content'].str.contains('스포일러가')]

#### csv 저장

In [32]:
n192021.to_csv('./data/total/Naver_review_192021.csv',index=False)

In [32]:
meta192021.to_csv('meta192021.csv',index=False)

In [33]:
len(n192021)

258405

In [34]:
movie_all = pd.merge(meta192021, n192021
    , how = 'inner'
    , on = ("title","release_date"))

In [35]:
len(n192021)

258405

In [36]:
movie_all = movie_all.drop_duplicates().reset_index(drop=True)
movie_all.to_csv('./data/total/Naver_review_192021.csv', index=False)

---

## 중복값 제거
* 중복된 리뷰가 수집된 경우 drop

In [46]:
movie_alldrop = movie_all.drop_duplicates().reset_index(drop=True)

In [49]:
movie_alldrop.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 235785 entries, 0 to 235784
Data columns (total 14 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   title         235785 non-null  object
 1   genre         235785 non-null  object
 2   nation        235785 non-null  object
 3   running_time  235785 non-null  object
 4   release_date  235785 non-null  object
 5   director      235785 non-null  object
 6   actor         235785 non-null  object
 7   grade         235785 non-null  object
 8   summary       235785 non-null  object
 9   poster        235785 non-null  object
 10  score         235785 non-null  int64 
 11  content       235785 non-null  object
 12  write_date    235785 non-null  object
 13  source_site   235785 non-null  object
dtypes: int64(1), object(13)
memory usage: 25.2+ MB


#### 리뷰 X 영화정보 merge : 
데이터 총 `234785개`


In [111]:
movie_alldrop[movie_alldrop['title']=='기생충'].head(20)

,title,genre,nation,running_time,release_date,director,actor,grade,summary,poster,score,content,write_date,source_site,pre_content
14556,기생충,드라마,한국,131분,2019 .05.30 개봉,봉준호,"송강호(기택), 이선균(동익), 조여정(연교)",15세 관람가,전원백수로 살 길 막막하지만 사이는 좋은 기택(송강호) 가족.\n 장남 기우(최우식...,https://movie-phinf.pstatic.net/20190528_36/15...,10,"비에 젖지 않는 고급 장난감 텐트와, 비에 젖다 못해 잠겨버리는 반지하 가구",2019.05.30 15:58,naver,
14557,기생충,드라마,한국,131분,2019 .05.30 개봉,봉준호,"송강호(기택), 이선균(동익), 조여정(연교)",15세 관람가,전원백수로 살 길 막막하지만 사이는 좋은 기택(송강호) 가족.\n 장남 기우(최우식...,https://movie-phinf.pstatic.net/20190528_36/15...,10,최근 본 영화중 가장 충격적이었음... 근데 보니까 15세말고 19세 걸어야될것같던데..,2019.05.30 12:25,naver,15 19
14558,기생충,드라마,한국,131분,2019 .05.30 개봉,봉준호,"송강호(기택), 이선균(동익), 조여정(연교)",15세 관람가,전원백수로 살 길 막막하지만 사이는 좋은 기택(송강호) 가족.\n 장남 기우(최우식...,https://movie-phinf.pstatic.net/20190528_36/15...,10,"지하철이라는 단어가 언급되는 순간, 대다수의 관객은 자신이 어디에 이입할 지를 안다.",2019.05.30 14:41,naver,
14559,기생충,드라마,한국,131분,2019 .05.30 개봉,봉준호,"송강호(기택), 이선균(동익), 조여정(연교)",15세 관람가,전원백수로 살 길 막막하지만 사이는 좋은 기택(송강호) 가족.\n 장남 기우(최우식...,https://movie-phinf.pstatic.net/20190528_36/15...,10,전 가정부가 집 벨 누를 때 부터 이 영화는 장르가 바뀐다... 역대급 꿀잼영화,2019.05.30 16:28,naver,
14560,기생충,드라마,한국,131분,2019 .05.30 개봉,봉준호,"송강호(기택), 이선균(동익), 조여정(연교)",15세 관람가,전원백수로 살 길 막막하지만 사이는 좋은 기택(송강호) 가족.\n 장남 기우(최우식...,https://movie-phinf.pstatic.net/20190528_36/15...,10,황금종려상 수상작을 자막 없이 볼 수 있다는 것 자체로 좋다.,2019.05.30 15:13,naver,
14561,기생충,드라마,한국,131분,2019 .05.30 개봉,봉준호,"송강호(기택), 이선균(동익), 조여정(연교)",15세 관람가,전원백수로 살 길 막막하지만 사이는 좋은 기택(송강호) 가족.\n 장남 기우(최우식...,https://movie-phinf.pstatic.net/20190528_36/15...,9,반지하가 제일 아래인줄 알았다. 내려가보니 지하도 있었다.,2019.05.31 01:41,naver,
14562,기생충,드라마,한국,131분,2019 .05.30 개봉,봉준호,"송강호(기택), 이선균(동익), 조여정(연교)",15세 관람가,전원백수로 살 길 막막하지만 사이는 좋은 기택(송강호) 가족.\n 장남 기우(최우식...,https://movie-phinf.pstatic.net/20190528_36/15...,8,뭔가 모를 불쾌한 영화였다. 영화가 불쾌하다는게 아니라 보는 내내 가슴에 뭔가 영화...,2019.05.30 10:50,naver,
14563,기생충,드라마,한국,131분,2019 .05.30 개봉,봉준호,"송강호(기택), 이선균(동익), 조여정(연교)",15세 관람가,전원백수로 살 길 막막하지만 사이는 좋은 기택(송강호) 가족.\n 장남 기우(최우식...,https://movie-phinf.pstatic.net/20190528_36/15...,8,나 또한 누군가의 사람 사는 냄새에 얼굴 찌푸리진 않았었나..,2019.05.30 15:11,naver,
14564,기생충,드라마,한국,131분,2019 .05.30 개봉,봉준호,"송강호(기택), 이선균(동익), 조여정(연교)",15세 관람가,전원백수로 살 길 막막하지만 사이는 좋은 기택(송강호) 가족.\n 장남 기우(최우식...,https://movie-phinf.pstatic.net/20190528_36/15...,10,"누군가의 냄새를 맡고, 평가하고, 묘사할 수 있는 것 또한 권력. 냄새로 서로를 알...",2019.05.30 12:05,naver,
14565,기생충,드라마,한국,131분,2019 .05.30 개봉,봉준호,"송강호(기택), 이선균(동익), 조여정(연교)",15세 관람가,전원백수로 살 길 막막하지만 사이는 좋은 기택(송강호) 가족.\n 장남 기우(최우식...,https://movie-phinf.pstatic.net/20190528_36/15...,10,박서준이 잘못했네ㅋㅋㅋㅋ,2019.05.30 15:01,naver,


In [110]:
movie_alldrop[movie_alldrop['title']=='기생충'].tail(20)

,title,genre,nation,running_time,release_date,director,actor,grade,summary,poster,score,content,write_date,source_site,pre_content
15018,기생충,드라마,한국,131분,2019 .05.30 개봉,봉준호,"송강호(기택), 이선균(동익), 조여정(연교)",15세 관람가,전원백수로 살 길 막막하지만 사이는 좋은 기택(송강호) 가족.\n 장남 기우(최우식...,https://movie-phinf.pstatic.net/20190528_36/15...,10,다른 어떤배우보다가정부 이정은씨 연기가 최고인듯,2019.05.31 08:38,naver,
15019,기생충,드라마,한국,131분,2019 .05.30 개봉,봉준호,"송강호(기택), 이선균(동익), 조여정(연교)",15세 관람가,전원백수로 살 길 막막하지만 사이는 좋은 기택(송강호) 가족.\n 장남 기우(최우식...,https://movie-phinf.pstatic.net/20190528_36/15...,6,기분나쁜 영화 인듯하네요,2019.06.06 20:32,naver,
15020,기생충,드라마,한국,131분,2019 .05.30 개봉,봉준호,"송강호(기택), 이선균(동익), 조여정(연교)",15세 관람가,전원백수로 살 길 막막하지만 사이는 좋은 기택(송강호) 가족.\n 장남 기우(최우식...,https://movie-phinf.pstatic.net/20190528_36/15...,10,무슨 말이 필요한가요. 명작!,2021.05.11 21:57,naver,
15021,기생충,드라마,한국,131분,2019 .05.30 개봉,봉준호,"송강호(기택), 이선균(동익), 조여정(연교)",15세 관람가,전원백수로 살 길 막막하지만 사이는 좋은 기택(송강호) 가족.\n 장남 기우(최우식...,https://movie-phinf.pstatic.net/20190528_36/15...,10,"간간히 웃으면서 봤지만 영화가 계속 될수록 마음이 무거워졌다. 반지하, 지하로 표...",2019.06.15 16:03,naver,
15022,기생충,드라마,한국,131분,2019 .05.30 개봉,봉준호,"송강호(기택), 이선균(동익), 조여정(연교)",15세 관람가,전원백수로 살 길 막막하지만 사이는 좋은 기택(송강호) 가족.\n 장남 기우(최우식...,https://movie-phinf.pstatic.net/20190528_36/15...,10,으잉?!했던. 지하에 사시는 아저씨가 너무 무서워요.,2019.06.10 17:39,naver,
15023,기생충,드라마,한국,131분,2019 .05.30 개봉,봉준호,"송강호(기택), 이선균(동익), 조여정(연교)",15세 관람가,전원백수로 살 길 막막하지만 사이는 좋은 기택(송강호) 가족.\n 장남 기우(최우식...,https://movie-phinf.pstatic.net/20190528_36/15...,5,다들 있어보이게 말들 써놨는데 걍 막장영화인듯,2019.06.07 14:42,naver,
15024,기생충,드라마,한국,131분,2019 .05.30 개봉,봉준호,"송강호(기택), 이선균(동익), 조여정(연교)",15세 관람가,전원백수로 살 길 막막하지만 사이는 좋은 기택(송강호) 가족.\n 장남 기우(최우식...,https://movie-phinf.pstatic.net/20190528_36/15...,1,대만 카스테라라는 소재를 사용함으로써 아팠던 기억을 다시 떠올리는 것만 해도 힘들 ...,2019.06.14 03:10,naver,
15025,기생충,드라마,한국,131분,2019 .05.30 개봉,봉준호,"송강호(기택), 이선균(동익), 조여정(연교)",15세 관람가,전원백수로 살 길 막막하지만 사이는 좋은 기택(송강호) 가족.\n 장남 기우(최우식...,https://movie-phinf.pstatic.net/20190528_36/15...,1,가난해본적 없는 사람이 그린 가난의 초상화. 기대에 차 보고나면 모두의 머리속에 드...,2019.06.05 12:48,naver,
15026,기생충,드라마,한국,131분,2019 .05.30 개봉,봉준호,"송강호(기택), 이선균(동익), 조여정(연교)",15세 관람가,전원백수로 살 길 막막하지만 사이는 좋은 기택(송강호) 가족.\n 장남 기우(최우식...,https://movie-phinf.pstatic.net/20190528_36/15...,1,영화저널리스트들과 세계각국의 영화관계자들께 말하고 싶네요 재미없고 감동없고 내용없고~~,2019.06.03 19:03,naver,
15027,기생충,드라마,한국,131분,2019 .05.30 개봉,봉준호,"송강호(기택), 이선균(동익), 조여정(연교)",15세 관람가,전원백수로 살 길 막막하지만 사이는 좋은 기택(송강호) 가족.\n 장남 기우(최우식...,https://movie-phinf.pstatic.net/20190528_36/15...,1,사람 가치는 고귀하고 무한합니다. ㅠㅠ 영화 보면서 사람이 너무 이상하게 묘사되네요,2019.06.03 12:16,naver,


In [50]:
movie_alldrop

,title,genre,nation,running_time,release_date,director,actor,grade,summary,poster,score,content,write_date,source_site
0,0.0MHz,공포,한국,102분,2019 .05.29 개봉,유선동,"정은지(소희), 이성열(상엽), 최윤영(윤정)",15세 관람가,"에디슨의 마지막 발명품, 유령 탐지기?\n “증명할 수 없는 미스터리란 없다”\n ...",https://movie-phinf.pstatic.net/20190517_285/1...,1,와 정말 무서운집 수준의 공포영화가 한국에 다시 나왔네요. 꼭 보러 가세요.착석하신...,2019.05.29 13:30,naver
1,0.0MHz,공포,한국,102분,2019 .05.29 개봉,유선동,"정은지(소희), 이성열(상엽), 최윤영(윤정)",15세 관람가,"에디슨의 마지막 발명품, 유령 탐지기?\n “증명할 수 없는 미스터리란 없다”\n ...",https://movie-phinf.pstatic.net/20190517_285/1...,10,"내용 구성이나 연출, cg는 저예산 영화 정도의 수준이었지만 정은지가 연기를 잘한다",2019.05.29 10:58,naver
2,0.0MHz,공포,한국,102분,2019 .05.29 개봉,유선동,"정은지(소희), 이성열(상엽), 최윤영(윤정)",15세 관람가,"에디슨의 마지막 발명품, 유령 탐지기?\n “증명할 수 없는 미스터리란 없다”\n ...",https://movie-phinf.pstatic.net/20190517_285/1...,2,상엽이 형은 어쩌다가 어릴때 죽었는지 다리 피흘리는 여고생 연관있을줄 알았는데 연관...,2019.05.29 20:15,naver
3,0.0MHz,공포,한국,102분,2019 .05.29 개봉,유선동,"정은지(소희), 이성열(상엽), 최윤영(윤정)",15세 관람가,"에디슨의 마지막 발명품, 유령 탐지기?\n “증명할 수 없는 미스터리란 없다”\n ...",https://movie-phinf.pstatic.net/20190517_285/1...,10,정은지 연기 소름...,2019.05.29 12:21,naver
4,0.0MHz,공포,한국,102분,2019 .05.29 개봉,유선동,"정은지(소희), 이성열(상엽), 최윤영(윤정)",15세 관람가,"에디슨의 마지막 발명품, 유령 탐지기?\n “증명할 수 없는 미스터리란 없다”\n ...",https://movie-phinf.pstatic.net/20190517_285/1...,6,일단 정은지는 완전 합격점. 최윤영도 좋았다. 여배우 둘 빼면 에휴...게다가 초반...,2019.05.29 11:34,naver
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235780,혼자 사는 사람들,드라마,한국,90분,2021 .05.19 개봉,홍성은,"공승연(진아), 정다은(수진)",12세 관람가,"집에서도 밖에서도 늘 혼자가 편한 진아.\n 사람들은 자꾸 말을 걸어오지만, 진아는...",https://movie-phinf.pstatic.net/20210506_173/1...,10,ㅅㅂ 돈도 ㅈ도 안주는데 ㅈ같ㅇ 영화다. 참는것도 한계가있지 갑질하는새끼들은 지출신...,2021.08.22 04:37,naver
235781,혼자 사는 사람들,드라마,한국,90분,2021 .05.19 개봉,홍성은,"공승연(진아), 정다은(수진)",12세 관람가,"집에서도 밖에서도 늘 혼자가 편한 진아.\n 사람들은 자꾸 말을 걸어오지만, 진아는...",https://movie-phinf.pstatic.net/20210506_173/1...,10,"바쁜 현대인들 삶과 , 삶에 대해 전하고자 하시는 내용이 너무 잘 표현된거 같다는 ...",2021.08.22 03:30,naver
235782,혼자 사는 사람들,드라마,한국,90분,2021 .05.19 개봉,홍성은,"공승연(진아), 정다은(수진)",12세 관람가,"집에서도 밖에서도 늘 혼자가 편한 진아.\n 사람들은 자꾸 말을 걸어오지만, 진아는...",https://movie-phinf.pstatic.net/20210506_173/1...,7,지독하게 현실감이 넘쳐서 영화 같지가 않았다.그리고 진아가 누굴까 하고 검색했는데 ...,2021.08.22 02:51,naver
235783,혼자 사는 사람들,드라마,한국,90분,2021 .05.19 개봉,홍성은,"공승연(진아), 정다은(수진)",12세 관람가,"집에서도 밖에서도 늘 혼자가 편한 진아.\n 사람들은 자꾸 말을 걸어오지만, 진아는...",https://movie-phinf.pstatic.net/20210506_173/1...,10,두렵고 무서운 영화에요,2021.08.22 02:41,naver


# 영화 장르 정리
movie['genre'].unique()
* 공포
* 멜로/로맨스
* 드라마
* 범죄
* 가족
* 코미디
* 액션
* 스릴러
* 애니메이션
* 모험
* 미스터리
* SF
* 전쟁
* 판타지
* 서스펜스
* 뮤지컬
* 블랙코미디

# 불용어 설정

* 한국불용어 100 + 영화, '진짜', '많이', '정말', '그냥', '그', '애', '생각', '완전', '편','내용','중간', '꼭','편','상미', '모두' , '거', 
https://bab2min.tistory.com/544#google_vignette

In [61]:
len(stop_words)

124

In [60]:
stop_words = list(set(stop_words))

In [57]:
stop_words= ['또','영화','분','꼭', '하', '받', '않', '말', '알','진짜', '많이', '정말', '그냥', '그', '애', '생각', '완전', '편','내용',
             '중간', '꼭','편','상미', '모두' , '거', 
 '더', '가', '뭐', '그렇', '말하', '중', '때', '되', '암', '보', '있', '개', '볼','일',
             '안','진짜', '듯', '위하', '대하', '년', '쫌', '위', '없', '주', '같', '나', '정말',
 '지', '걸', '못','좀', '그', '왜','때문', '임', '게', '것', '의', '사람', '이','대한',
             '수','이','있','하','것','들','그','되','수','보','않','나','사람','주','아니',
             '등',
             '같','우리','때','년','가','한','지','대하','오','말','일','그렇','위하','때문',
             '그것','두','말하','알','그러나','그런데','그래도','그래서','받','못하','일',
             '그런','또','문제','더','사회','많','좋','크','중','나오','가지','씨','시키',
             '만들','지금','생각하','그러','속','하나','집','살','모르','적','월','데','자신',
             '안','어떤','내','경우','명','생각','시간','그녀','다시','이런','앞','보이','번',
             '나','다른','어떻','여자','남자','개','전','들','사실','이렇','점','싶','말','정도',
             '좀','원','잘','통하','소리','놓','영화','진짜', '많이', '정말', '그냥', '그', '애',
             '생각', '완전', '편','내용','중간', '꼭','편','상미', '모두' , '거']


---

# 장르 통합 리뷰

In [91]:
allcontent = []
for words in tqdm(movie_alldrop['content']):
    # word_tokenize을 통해서 Tokenizer를 진행해줍니다. 
    word_tokens = word_tokenize(words)
    # re.sub('[^a-z0-9]+', ' ', str(word_tokens)) 명령어를 통해서 특수문자를 모두 제거해줍니다. 
    sentence = re.sub('[^A-Za-z0-9]+', ' ', str(word_tokens))
    # 양끝에 존재하는 공백을 제거해줍니다. 
    sentence = sentence.strip()

    # 문장의 Tokenizer를 진행해줍니다. 
    sentence_tokens = word_tokenize(sentence)
    result = ''
    for token in sentence_tokens: 
        if token not in stop_words:
            result += ' ' + token 
    result = result.strip().lower()
    allcontent.append(result)

  0%|          | 0/235785 [00:00<?, ?it/s]

In [92]:
movie_alldrop['pre_content'] = allcontent

In [93]:
from collections import namedtuple

# Doc2Vec 입력형태를 맞춰줍니다. 
# TaggedDocument 형태로 ('영화 내용', '제목') 을 받습니다. 
## 이후, TaggedDocument에 모든 문서에 대해서 '영화 내용', '제목'을 넣어줍니다. 
agg = movie_alldrop[['title', 'pre_content']]
TaggedDocument = namedtuple('TaggedDocument', 'words tags')
tagged_train_docs = [TaggedDocument((c), [d]) for d, c in agg[['title', 'pre_content']].values]

In [95]:
doc_vectorizer = doc2vec.Doc2Vec(
    dm=1,            # PV-DBOW / default 1
    #dbow_words=1,    # w2v simultaneous with DBOW d2v / default 0
    window=10,        # distance between the predicted word and context words
    vector_size=300,        # vector size
    alpha=0.025,     # learning-rate
    seed=1234,
    #min_count=5,    # ignore with freq lower
    min_alpha=0.025, # min learning-rate
    #workers=4,   # multi cpu
    #hs = 1,          # hierar chical softmax / default 0
    negative = 5   # negative sampling / default 5
)

In [96]:
# 위에서 만든 tagged_train_docs으로 사전을 만들어줍니다. 
doc_vectorizer.build_vocab(tagged_train_docs)
print(str(doc_vectorizer))

Doc2Vec(dm/m,d300,n5,w10,mc5,s0.001,t3)


In [97]:
# To save
doc_vectorizer.save('doc2vec_all.model')

In [98]:
# 벡터 문서 학습
from time import time

start = time()

for epoch in tqdm(range(30)):
    # Doc2Vec 학습을 진행하는데 Learning rate를 계속 감소해주면서 학습을 진행해줍니다. 
    doc_vectorizer.train(tagged_train_docs, total_examples=doc_vectorizer.corpus_count, epochs=30)
    doc_vectorizer.alpha -= 0.002 # decrease the learning rate
    doc_vectorizer.min_alpha = doc_vectorizer.alpha # fix the learning rate, no decay

end = time()
print("During Time: {}".format(end-start))

  0%|          | 0/30 [00:00<?, ?it/s]

During Time: 8826.567391395569


In [102]:
doc_vectorizer.docvecs.most_similar('기생충', topn=20)

[('프린세스 에미: 마법 책의 비밀', 0.6510905027389526),
 ('대환장파티', 0.6344231367111206),
 ('맨슨 패밀리', 0.6096524000167847),
 ('내 완벽한 남사친의 비밀', 0.6074991822242737),
 ('프리즌 브레이커스', 0.5989813804626465),
 ('영화로운 나날', 0.5903171896934509),
 ('보르도 우정여행', 0.5771888494491577),
 ('파이널리스트', 0.5678276419639587),
 ('프론테라스', 0.562434732913971),
 ('7월7일', 0.5503718256950378),
 ('체크 히어로', 0.5486385822296143),
 ('슈퍼문', 0.5434455275535583),
 ('더 배니싱', 0.5404161214828491),
 ('고골: 악령과의 전쟁', 0.5395044684410095),
 ('O.J. 심슨 사건 파일', 0.5385341048240662),
 ('구르는 수레바퀴', 0.5361682772636414),
 ('초대녀 2', 0.5360038876533508),
 ('그리고 우린 춤을 추었다', 0.535868227481842),
 ('기묘한 이야기: 리플레이', 0.5341373682022095),
 ('얼굴들', 0.5340752601623535)]

In [106]:
doc_vectorizer.docvecs.most_similar('보스 베이비 2', topn=20)

[('그것: 두 번째 이야기', 0.576075553894043),
 ('트롤: 월드 투어', 0.5725052952766418),
 ('스웨그', 0.5673669576644897),
 ('초미의 관심사', 0.5517657995223999),
 ('컨저링 3: 악마가 시켰다', 0.5465413331985474),
 ('닥터 두리틀', 0.5164274573326111),
 ('범털2 : 쩐의 전쟁', 0.513810396194458),
 ('겨울왕국 2', 0.509134829044342),
 ('콰이어트 플레이스 2', 0.501354992389679),
 ('데드풀2: 순한 맛', 0.4988044798374176),
 ('오시로가의 비밀', 0.49445825815200806),
 ('날씨의 아이', 0.49288317561149597),
 ('극장판 짱구는 못말려: 격돌! 낙서왕국과 얼추 네 명의 용사들', 0.4908711314201355),
 ('걸후드', 0.4896414875984192),
 ('나의 흑역사 로맨티카', 0.48686811327934265),
 ('메모리즈', 0.4853954613208771),
 ('스네이크 아이즈: 지.아이.조', 0.48079225420951843),
 ('피닉스', 0.47633570432662964),
 ('완벽한 그녀의 비밀', 0.4735615849494934),
 ('신의 한 수: 귀수편', 0.47197017073631287)]

---

# 장르 : 공포

## → 장르가 공포를 포함하는 모든 영화
## 공포'만' 해당X

In [64]:
horror = movie_alldrop[movie_alldrop['genre'].str.contains('공포')].reset_index(drop=True)

In [ ]:
from gensim.models import doc2vec
# 사전 구축
doc_vectorizer = doc2vec.Doc2Vec(size=300, alpha=0.025, min_alpha=0.025, seed=1234)
doc_vectorizer.build_vocab(tagged_train_docs)

In [62]:
from gensim.models import doc2vec

In [63]:
from nltk.corpus import stopwords 
from tqdm.notebook import tqdm
from nltk.tokenize import word_tokenize, sent_tokenize

---

# 공포영화만 유사도 측정하기

In [65]:
hcontent = []
for words in tqdm(horror['content']):
    # word_tokenize을 통해서 Tokenizer를 진행해줍니다. 
    word_tokens = word_tokenize(words)
    # re.sub('[^a-z0-9]+', ' ', str(word_tokens)) 명령어를 통해서 특수문자를 모두 제거해줍니다. 
    sentence = re.sub('[^A-Za-z0-9]+', ' ', str(word_tokens))
    # 양끝에 존재하는 공백을 제거해줍니다. 
    sentence = sentence.strip()

    # 문장의 Tokenizer를 진행해줍니다. 
    sentence_tokens = word_tokenize(sentence)
    result = ''
    for token in sentence_tokens: 
        if token not in stop_words:
            result += ' ' + token 
    result = result.strip().lower()
    hcontent.append(result)

  0%|          | 0/22291 [00:00<?, ?it/s]

In [ ]:
from gensim.models import doc2vec
# 사전 구축
doc_vectorizer = doc2vec.Doc2Vec(size=300, alpha=0.025, min_alpha=0.025, seed=1234)
doc_vectorizer.build_vocab(tagged_train_docs)

In [66]:
horror['pre_content'] = hcontent

In [ ]:
## 장르 X
## 줄거리 + 리뷰 

In [67]:
doc_vectorizer = doc2vec.Doc2Vec(
    dm=1,            # PV-DBOW / default 1
    #dbow_words=1,    # w2v simultaneous with DBOW d2v / default 0
    window=10,        # distance between the predicted word and context words
    vector_size=300,        # vector size
    alpha=0.025,     # learning-rate
    seed=1234,
    #min_count=5,    # ignore with freq lower
    min_alpha=0.025, # min learning-rate
    #workers=4,   # multi cpu
    #hs = 1,          # hierar chical softmax / default 0
    negative = 5   # negative sampling / default 5
)

In [68]:
from collections import namedtuple

# Doc2Vec 입력형태를 맞춰줍니다. 
# TaggedDocument 형태로 ('영화 내용', '제목') 을 받습니다. 
## 이후, TaggedDocument에 모든 문서에 대해서 '영화 내용', '제목'을 넣어줍니다. 
agg = horror[['title', 'pre_content']]
TaggedDocument = namedtuple('TaggedDocument', 'words tags')
tagged_train_docs = [TaggedDocument((c), [d]) for d, c in agg[['title', 'pre_content']].values]

In [69]:
# 위에서 만든 tagged_train_docs으로 사전을 만들어줍니다. 
doc_vectorizer.build_vocab(tagged_train_docs)
print(str(doc_vectorizer))

Doc2Vec(dm/m,d300,n5,w10,mc5,s0.001,t3)


In [79]:
# To save
doc_vectorizer.save('doc2vec.model')

In [75]:
# 벡터 문서 학습
from time import time

start = time()

for epoch in tqdm(range(30)):
    # Doc2Vec 학습을 진행하는데 Learning rate를 계속 감소해주면서 학습을 진행해줍니다. 
    doc_vectorizer.train(tagged_train_docs, total_examples=doc_vectorizer.corpus_count, epochs=30)
    doc_vectorizer.alpha -= 0.002 # decrease the learning rate
    doc_vectorizer.min_alpha = doc_vectorizer.alpha # fix the learning rate, no decay

end = time()
print("During Time: {}".format(end-start))

  0%|          | 0/30 [00:00<?, ?it/s]

During Time: 605.9310328960419


In [76]:
horror['title'].unique()

array(['0.0MHz', '47미터 2', '공포의 묘지', '귀신의 향기', '그레이브 하우스', '나이트메어 시네마',
       '노크: 낯선 그림자', '늑대의 아이들', '다커 나이트', '닥터 킬러 패밀리', '더 바이 바이 맨',
       '더 보이', '더 빌리지', '더 스쿨', '더하우스', '데드 돈 다이', '디 엔드?',
       '레드 스프링 아포칼립스', '로드킬', '룩 어웨이', '매리', '매트리악', '맨디', '미드소마',
       '미이라 리턴즈', '베스와 베라', '변신', '보어 살인 멧돼지', '사일런스', '사자', '사탄의 인형',
       '사탄의 인형: 상속 받은 저주', '살인마 잭의 집', '악령의 캠핑카', '암전',
       '애나벨 : 잊혀지지 않는 선율', '애나벨 집으로', '어스', '언데드 인 커밍', '요로나의 저주',
       '위치 오브 하우스', '이스케이프 룸', '인면어: 저주의 시작', '좀비 : 죽은 자들의 도시',
       '좀비랜드: 더블 탭', '카르마', '카리스마타 살인', '카메라를 멈추면 안 돼! 스핀오프 : 할리우드 대작전',
       '카운트다운', '캐빈 오브 더 컨저링', '컨벤트', '크로우 라이징', '파라노말 액티비티 2019',
       '파이와켓: 죽음의 주문', '폐교', '폴라로이드', '프로디지', '해피 데스데이 2 유', '헌팅 오브 힐하우스',
       '검은 사제들: 지옥의 문', '겟 아웃 얼라이브', '고스트 오브 워', '그곳', '그집', '기기괴괴 성형수',
       '기묘한 이야기: 리플레이', '나이트메어 라디오', '나이트메어 서커스', '나치 오버로드', '뉴 뮤턴트',
       '더 렌탈: 소리없는 감시자', '더 보이 2: 돌아온 브람스', '더 터닝', '데빌 인사이드 미', '돈 스피크',
       '디어스킨', '라비드', '런 런 런', '레벨 16', '맨슨

In [83]:
doc_vectorizer.docvecs.most_similar(positive=['컨저링: 귀신의 집', '공포의 묘지'], negative=['47미터 2'], topn=20)

[('인면어: 저주의 시작', 0.5635553002357483), ('세인트 아가타', 0.5156008005142212), ('자매의 공동묘지', 0.4939689338207245), ('매트리악', 0.4887956976890564), ('맨디', 0.48686933517456055), ('링: 더 라스트 챕터', 0.46589788794517517), ('데빌 인사이드 미', 0.46528691053390503), ('더 스쿨', 0.46294263005256653), ('레드 스프링 아포칼립스', 0.46280795335769653), ('헌티드 파크', 0.46124446392059326), ('돈 스피크', 0.45582059025764465), ('존 도우', 0.4442197382450104), ('목격자 : 눈이 없는 아이', 0.4434628486633301), ('더 나이트', 0.44225889444351196), ('악플러 죽이기', 0.44203585386276245), ('실종: 사라진 사람들', 0.42329758405685425), ('요가학원:죽음의쿤달리니', 0.42286810278892517), ('악령의 캠핑카', 0.41867345571517944), ('런 런 런', 0.41380488872528076), ('피어썸', 0.40190389752388)]

In [78]:
doc_vectorizer.docvecs.most_similar('컨저링 3: 악마가 시켰다', topn=20)

[('아이 씨 유', 0.650468647480011),
 ('실종: 사라진 사람들', 0.6181316375732422),
 ('카르마', 0.6146055459976196),
 ('그곳', 0.6142624020576477),
 ('애나벨 : 잊혀지지 않는 선율', 0.6089791059494019),
 ('컬트 걸', 0.6086001396179199),
 ('캐빈 오브 더 컨저링', 0.6045429110527039),
 ('신 강시선생', 0.6001917719841003),
 ('해피 데스데이 2 유', 0.5811768770217896),
 ('피어 오브 레인', 0.5806698203086853),
 ('더 배니싱', 0.5653866529464722),
 ('47미터 2', 0.5523381233215332),
 ('위자 크래프트', 0.5474677681922913),
 ('파라노말 액티비티 2019', 0.5408865213394165),
 ('살인 매뉴얼', 0.5378244519233704),
 ('크로우 라이징', 0.5057649612426758),
 ('리틀 조', 0.5034528374671936),
 ('더 빌리지', 0.4987199902534485),
 ('닥터 킬러 패밀리', 0.4970897138118744),
 ('고스트 오브 워', 0.4949326515197754)]

In [89]:
doc_vectorizer.docvecs.most_similar('파라노말 액티비티 2019', topn=20)

[('블러드: 궁금한 이야기', 0.9891312122344971),
 ('캐빈 오브 더 컨저링', 0.8981232643127441),
 ('리틀 조', 0.863559365272522),
 ('기묘한 이야기: 리플레이', 0.8523256182670593),
 ('카르마', 0.8510911464691162),
 ('애나벨 : 잊혀지지 않는 선율', 0.8502852916717529),
 ('맨슨 패밀리', 0.8493539094924927),
 ('그곳', 0.8033229112625122),
 ('아이 씨 유', 0.7965874671936035),
 ('신 강시선생', 0.7965527176856995),
 ('더 배니싱', 0.7948595285415649),
 ('더 빌리지', 0.7891238927841187),
 ('피어 오브 레인', 0.7856281995773315),
 ('위자 크래프트', 0.7809276580810547),
 ('살인 매뉴얼', 0.7757599353790283),
 ('크로우 라이징', 0.7675133943557739),
 ('나이트메어 서커스', 0.7662829160690308),
 ('사라져버린', 0.7643951773643494),
 ('닥터 킬러 패밀리', 0.7599900364875793),
 ('컬트 걸', 0.757792055606842)]